**bronze lti facility transaction**

In [0]:
from pyspark.sql import functions as F

raw = (
  spark.table("tp_finance.raw.github_sheet_rows")
    .where(F.col("sheet_name") == "Draw_Repay")
)

m = F.from_json("row_json", "map<string,string>")

tx = (
  raw.select(
      F.col("source_system"),
      F.col("source_path"),
      F.col("file_sha256"),
      F.col("load_date"),
      F.col("source_modified_ts"),
      F.col("ingestion_run_id"),
      F.col("load_ts"),
      m.alias("m")
  )
  .select(
      F.col("m")["customer_id"].alias("customer_id"),
      F.col("m")["contract_id"].alias("contract_id"),
      F.col("m")["facility_id"].alias("facility_id"),
      F.to_date(F.col("m")["transaction_date"]).alias("transaction_date"),
      F.upper(F.col("m")["transaction_type"]).alias("transaction_type"),
      F.col("m")["amount"].cast("decimal(18,2)").alias("amount"),
      F.col("m")["reference"].alias("reference"),
      F.col("m")["currency"].alias("currency"),

      "source_system","source_path","file_sha256","load_date","source_modified_ts","ingestion_run_id","load_ts"
  )
  .withColumn(
      "record_hash",
      F.sha2(F.concat_ws("||",
        F.coalesce(F.col("customer_id"), F.lit("")),
        F.coalesce(F.col("contract_id"), F.lit("")),
        F.coalesce(F.col("facility_id"), F.lit("")),
        F.coalesce(F.col("transaction_date").cast("string"), F.lit("")),
        F.coalesce(F.col("transaction_type"), F.lit("")),
        F.coalesce(F.col("amount").cast("string"), F.lit("")),
        F.coalesce(F.col("reference"), F.lit("")),
        F.coalesce(F.col("file_sha256"), F.lit(""))
      ), 256)
  )
)

# Basic DQ filter (optional but recommended)
tx_clean = tx.where(
    F.col("facility_id").isNotNull() &
    F.col("transaction_date").isNotNull() &
    F.col("amount").isNotNull() &
    (F.col("amount") > 0) &
    F.col("transaction_type").isin("DRAW","REPAY")
)

tx_clean.createOrReplaceTempView("stg_tx")

spark.sql("""
MERGE INTO tp_finance.bronze.lti_facility_transactions t
USING stg_tx s
ON  t.record_hash = s.record_hash
WHEN NOT MATCHED THEN INSERT *
""")

# This MERGE uses record_hash to prevent duplicates on reruns